In [ ]:
import pickle
import seaborn as sns
import networkx as nx
import collections 
import community
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy

# Construction du Graphe

In [ ]:
#Lecture du fichier des liens
df_edges = pd.read_csv("/home/pb19121/mydata/edgesVirgule.txt",sep =',',header = None)
df_edges.columns = ['FROM', 'TO']

In [3]:
df_edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14067887 entries, 0 to 14067886
Data columns (total 2 columns):
FROM    int64
TO      int64
dtypes: int64(2)
memory usage: 214.7 MB


###### vérification de l'exhaustivité des noeuds

In [4]:
#Lecture du fichier des noeuds
df_nodes = pd.read_csv("/home/pb19121/mydata/nodes.csv",header = None)
df_nodes.columns = ['NODE']

In [5]:
df_nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100199 entries, 0 to 100198
Data columns (total 1 columns):
NODE    100199 non-null int64
dtypes: int64(1)
memory usage: 782.9 KB


In [ ]:
pd.DataFrame(df_nodes.NODE.unique()).count()

In [ ]:
df_FROM =pd.DataFrame(df_edges.FROM.unique())
df_FROM.columns = ['NODE']
df_FROM.info()

In [ ]:
df_TO =pd.DataFrame(df_edges.TO.unique())
df_TO.columns = ['NODE']
df_TO.info()

In [ ]:
frames = [df_FROM, df_TO, df_nodes]

In [ ]:
df_union_nodes =pd.concat(frames)

In [ ]:
pd.DataFrame(df_union_nodes.NODE.unique()).count()

> Tous les noeuds ne figuraient pas dans le fichiers nodes

##### Suppression des arretes dont les noeuds sont absents du fichiers nodes.csv

In [ ]:
liste = df_nodes.NODE

In [ ]:
#df_edges_isnode = df_edges[((df_edges['FROM'].isin(df_nodes.NODE)) or (df_edges['TO'].isin(df_nodes.NODE)))]
df_edges_isnode = df_edges[(df_edges['FROM'].isin(liste) )& (df_edges['TO'].isin(liste))]

In [ ]:
df_edges_isnode.info()

In [ ]:
DG = nx.DiGraph()

In [ ]:
for  item, row in df_edges_isnode.iterrows():
    DG.add_edge(row['FROM'],row['TO'], weight=1)    

In [ ]:
#_SAUVEGARDE DU GRAPHE DANS UN FICHIER
nx.write_edgelist(DG, "/home/pb19121/mydata/data.edgelist")

In [6]:
#_LECTURE DU GRAPHE DEPUIS UN FICHIER
DG = nx.read_edgelist("/home/pb19121/mydata/data.edgelist",create_using=nx.DiGraph())

In [9]:
DG = nx.freeze(DG)

In [10]:
G = DG.to_undirected()

In [11]:
G = nx.freeze(G)

In [ ]:
#_POUR PLUS TARD
import json
from networkx.readwrite import json_graph
node_link_data=json_graph.node_link_data(DG)
#s_node_link_data=json.dumps(data)
adjacency_data = json_graph.adjacency_data(DG)
#s_adjacency_data=json.dumps(data_adj)
with open('/home/pb19121/mydata/adjacency_data.json', 'w') as outfile: 
    json.dump(adjacency_data, outfile)
with open('/home/pb19121/mydata/node_link_data.json', 'w') as outfile:  
    json.dump(node_link_data, outfile)

# Analyse topologique

##### - Informations générales

In [7]:
print(nx.info(DG))

Name: 
Type: DiGraph
Number of nodes: 100199
Number of edges: 981920
Average in degree:   9.7997
Average out degree:   9.7997


In [12]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 100199
Number of edges: 981920
Average degree:  19.5994


##### - Densité

In [13]:
print("densité de DG: %s" % nx.density(DG))

print("densité de G: %s" % nx.density(G))

densité de DG: 9.780333539378456e-05
densité de G: 0.00019560667078756912


> **_Commentaires_**

##### - Noeud

In [ ]:
#print ("degree (undirected) %s" % G.degree())   #ensemble des sommets
att_deg = G.degree()

type(deg)

In [ ]:
nx.set_node_attributes(G, 'degres', att_deg)

In [ ]:
#print ("betweenness centrality %s" % nx.betweenness_centrality(G))
betweenness = nx.betweenness_centrality(G)
#nx.set_node_attributes(G, 'betweenness_centrality', betweenness)

In [ ]:
#print("eccentricity: %s" % nx.eccentricity(G))
eccentricity =   nx.eccentricity(G)
#nx.set_node_attributes(G, 'eccentricity', eccentricity)

In [ ]:
#DG.nodes.data()

##### - Degrés 

In [ ]:
degree_sequence = sorted([d for n, d in DG.degree()], reverse=True)  # degree sequence
# print "Degree sequence", degree_sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:

plt.loglog(degree_sequence)


In [ ]:

cle, valeur = zip(*degreeCount.items())


In [ ]:
len(valeur)

In [ ]:
d ={'cle' : pd.Series(cle),'val' : pd.Series(valeur)}
df =pd.DataFrame(d)

In [ ]:
df.describe()

In [ ]:
g=sns.barplot(x="cle", y="val", data=df)


#sns.despine(offset=10, trim=True)

In [ ]:
g=sns.factorplot(x="cle",y="val", data=df,  size=3, aspect=5)
g.set_xticklabels(step=11, label="titre")
g.set_xticklabels(rotation=90)

In [ ]:
#print ("degree (normalized) %s" % nx.degree_centrality(G))
degree_normalized =  nx.degree_centrality(G)

In [ ]:
len(degree_normalized)

In [ ]:
#nx.set_node_attributes(G, 'degree_normalized', degree_normalized)

In [ ]:
d ={'deg' : pd.Series(deg),'cnt' : pd.Series(cnt)}
df =pd.DataFrame(d)

In [ ]:
max(degree_sequence)

In [ ]:
sequence = (d for n, d in G.degree())

In [ ]:
nx.is_graphical(sequence,method="hh")

In [ ]:
nx.is_graphical(sequence,method="eg")

##### -  Connexité

In [ ]:
number_connected_components = nx.number_connected_components(G)

number_connected_components

In [ ]:
DG.number_of_selfloops()

In [ ]:
giant = max(nx.connected_component_subgraphs(G), key=len)
type(giant)

In [ ]:
is_strongly_connected_DG = nx.is_strongly_connected(DG)
is_strongly_connected_DG

In [ ]:
is_weakly_connected_DG = nx.is_weakly_connected(DG)
is_weakly_connected_DG

In [ ]:
print(nx.info(giant))

In [ ]:
centres = nx.center(G)

In [ ]:
type(centres)

In [ ]:
len(centres)

In [ ]:
print("centre : %s" % nx.center(G))

In [ ]:
#print("péripherie : %s" % nx.periphery(G))
peripherie =  nx.periphery(G)

In [ ]:
type(peripherie)

##### -  Cliques

In [ ]:
#recherche des cliques
cl = list(nx.find_cliques(G))
#print ("cliques %s" % cl)

len(cl)

##### -  Condensation

In [ ]:
valeur_condensation = nx.condensation(DG)

In [ ]:
print(nx.info(valeur_condensation))

##### -  Plus courts chemins

In [ ]:
#plus courts chemins
pathlengths=[]

print("source vertex {target:length, }")
for v in G.nodes():
    spl=nx.single_source_shortest_path_length(G,v)
    #print('%s %s' % (v,spl))
    for p in spl.values():
        pathlengths.append(p)
#print('')
print("moyenne des plus court chemins %s" % (sum(pathlengths)/len(pathlengths)))

#distribution des plus courts chemins
dist={}
for p in pathlengths:
    if p in dist:
        dist[p]+=1
    else:
        dist[p]=1

#dist

> **_Commentaires_**

# Clustering 

# Recherche de communauté de Louvain

In [ ]:
# l'algorithme de Louvain cherche les communautes
part = community.best_partition(G)
values = [part.get(node) for node in G.nodes()]

In [ ]:
len(part)

In [ ]:
# valeur de la modularité
mod = community.modularity(part,G)
print("modularity: %0.5f"%mod)

In [ ]:
pd.DataFrame(values).to_csv('/home/pb19121/mydata/values_part_louvain.txt', sep = ';',index = True, header= True)

sns.distplot(values)

In [ ]:
# networkx propose une visualisation (reposant sur une disposition "spring")
# des noeuds, avec des couleurs correspondants aux groupes obtenus
plt.figure(figsize=(20,15))
nx.draw_spring(G, cmap = plt.get_cmap('jet'), node_color = values, node_size=10, with_labels=True)
plt.show()

# Analyse spectrale

In [ ]:
L = nx.normalized_laplacian_matrix(G)

In [ ]:
M = nx.to_scipy_sparse_matrix(G)

In [ ]:
I = nx.incidence_matrix(DG)

In [ ]:
print(I)

In [ ]:
A = nx.adjacency_matrix(DG)

In [ ]:
e = numpy.linalg.eigvals(L.A)

In [ ]:
print("Largest eigenvalue:", max(e))
print("Smallest eigenvalue:", min(e))

In [ ]:
plt.hist(e,bins=100) # histogram with 100 bins
plt.xlim(0,2)  # eigenvalues between 0 and 2
plt.show()

# Visualisation

In [ ]:
#visualisation
pos=nx.spring_layout(DG)

In [ ]:
nx.draw(G, pos, with_labels=False)
plt.show()

In [ ]:
print(M.todense())

# Divers

partition dendrogramme

In [ ]:
dendrogram = community.generate_dendrogram(G)

In [ ]:
len(community.partition_at_level(dendrogram, 2))

In [ ]:
for level in range(0, 2) :
    print("partition au niveau ", level, "est ", community.partition_at_level(dendrogram, level))  # NOQA

Riche club

In [ ]:
Gtest = nx.Graph([(5, 1), (5, 2), (1, 2), (1, 3), (1, 4), (4, 5)])
rc = nx.rich_club_coefficient(Gtest, normalized=False)

In [ ]:
rc = nx.rich_club_coefficient(G, normalized=False)

In [ ]:
len(rc)

In [ ]:
tG = nx.MultiDiGraph()
tG.add_edge(0,1,weight=2)
tG.add_edge(1,0)
tG.add_edge(2,2,weight=3)
tG.add_edge(2,2)
tS = nx.to_scipy_sparse_matrix(tG, nodelist=[0,1,2])
print(tS.todense())

In [ ]:
nodeliste = G.nodes().to

In [ ]:
nodeliste2 = [100000, 100016, 100018, 100587, 102358, 102445, 103095, 103808, 104275, 106630, 107018, 108442, 108460, 108463, 108479, 108837, 108871, 109519, 110820, 111476, 100001, 100007, 100011, 100062, 100075, 100086, 100096, 106358, 106378, 108506, 100002, 100004, 100005, 100012, 100017, 100019, 100025, 100054, 100065, 100066, 100074, 100080, 100082, 100091, 100095, 100099, 100102, 100552, 101464, 106103, 106333, 106373, 106759, 106770, 107140, 110767, 110794, 111488, 116056, 100006, 100008, 100048, 100053, 100089, 112172, 100015, 100037, 100045, 100094, 102425, 104912, 109759, 109762, 109765, 109766, 109775, 113623, 100034, 100077, 100078, 106317, 106352, 106383, 108456, 108477, 108505, 108510, 109764, 109947, 110762, 101383, 103947, 103956, 107315, 107680, 108559, 119459, 119498, 119502, 119503, 100009, 100030, 100033, 100040, 100050, 100067, 100069, 100070, 100083, 100085, 100097, 100105, 100107, 100543, 102200, 104017, 106332, 106342, 108438, 108441, 108443, 108457, 108458, 108462, 108475, 108487, 108490, 108491, 108492, 108494, 108513, 108522, 108525, 108528, 108538, 108541, 108542, 1000, 1001, 1003, 100010, 100027, 100098, 102533, 103885, 1030, 100061, 100063, 100766, 102391, 102455, 102963, 104637, 108903, 110749, 1169, 100101, 104225, 104233, 104244, 104266, 108330, 108807, 111449, 100013, 100021, 100022, 100039, 100041, 100042, 100046, 100047, 100049, 100588, 100657, 101546, 101910, 102291, 105500, 106228, 107305, 107311, 107321, 107332, 107340, 107349, 107358, 109905, 111866, 119464, 119478, 119481, 119494, 119505, 119506, 119514, 119518, 119523, 100029, 100058, 100064, 100071, 100073, 100079, 100104, 108444, 108484, 108500, 108509, 108546, 110826, 110839, 132245, 100023, 100035, 100057, 100068, 100088, 101941, 103713, 104221, 104222, 105134, 106683, 107147, 108320, 108455, 108465, 108467, 108468, 108470, 108471, 108473, 108474, 108480, 108481, 108485, 108518, 108524, 108545, 110795, 110814, 111474, 120461, 102049, 113022, 120465, 146263, 101722, 101778, 102481, 106318, 106362, 106476, 106640, 107595, 108266, 108453, 108459, 108508, 108877, 108883, 110663, 110669, 100535, 100888, 102584, 104022, 105133, 105166, 105914, 105926, 106344, 106379, 106521, 106753, 106757, 106763, 106767, 106771, 106772, 106780, 106781, 106796, 106845, 107497, 108440, 109749, 109750, 109782, 109931, 109952, 110043, 110061, 110619, 111452, 111473, 113325, 120281, 120282, 120289, 120293, 120480, 10002, 10041, 10045, 10051, 10057, 10066, 10074, 100793, 102616, 104087, 107829, 108406, 110172, 21111, 113618, 120291, 11367, 103701, 103758, 104037, 106083, 107708, 119489, 119511, 119521, 106331, 106364, 106381, 109326, 111536, 120535, 37659, 37690, 37699, 37700, 37709, 37730, 37747, 37762, 37794, 37802, 37816, 37827, 37841, 37842, 100538, 104630, 105143, 107952, 108523, 58864, 100026, 100542]

In [ ]:
adjacencyMatrix = nx.to_numpy_matrix(A,nodelist=nodeliste)

In [ ]:
import scipy as sp
G_test = nx.Graph([(1,3),(1,2),(4,2)])
A_test = nx.adjacency_matrix(G_test)
print(A_test)

A_test.setdiag(A_test.diagonal()*2)
print(A_test.todense())


In [ ]:
T = nx.to_numpy_recarray(DG, dtype=[('weight', int)])
print(T.weight)

In [ ]:
type(x)

In [ ]:
matrice = numpy.array([[1, 1, 1], [1, 3, 1], [1, 1, 1]])
im1 = plt.matshow(A.todense())
plt.colorbar(im1)
plt.show

In [ ]:
sns.heatmap(A.todense())


In [ ]:
print(Ase())

In [ ]:
print(A.transpose())

In [ ]:
print(M.todense())